In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from pathlib import Path
import matplotlib as mpl

from sdm_eurec4a.visulization import set_custom_rcParams, handler_map_alpha, adjust_lightness_array

default_colors = set_custom_rcParams()
dark_colors = adjust_lightness_array(default_colors, amount=0.5)

In [ ]:
dataset_path = Path(
    "/home/m/m301096/CLEO/data/output_v3.0/stationary_condensation/combined/eulerian_dataset_combined.nc"
)
ds = xr.open_dataset(dataset_path)
sub_cloud_layer = ds["gridbox"] < ds["max_gridbox"]
sub_cloud_layer

In [ ]:
# %%
precipitation = (
    60
    * 60
    / ds["time"].diff(dim="time").mean()
    / 20
    / 20
    * ds["mass_represented"].sel(time=slice(400, None)).sum(dim="radius_bins").mean("time")
)
evaporated_mass = 1 / 20 / 20 / 20 * ds["mass_difference"].sel(time=slice(400, None)).mean("time")
evaporated_fraction = (
    1
    / 20
    / 20
    / 20
    * (ds["mass_difference"] / ds["mass_represented"]).sel(time=slice(400, None)).mean("time")
)

evaporated_mass_all = 1 / 20 / 20 / 20 * ds["mass_difference"].sel(time=slice(400, None))

In [ ]:
mr = ds["mass_represented"].sel(time=slice(400, None))
mr_m = mr.mean(dim="radius_bins")
sk = (1 / len(mr["radius_bins"]) * ((mr - mr_m) ** 3).sum(dim="radius_bins")) / (
    1 / (len(mr["radius_bins"]) - 1) * ((mr - mr_m) ** 2).sum(dim="radius_bins") ** (3 / 2)
)

ds["mass_moment_2"] = sk
ds["mass_moment_2"].attrs[
    "long_name"
] = "Mass moment 2 as the skweness of the mass distribution along radius_bins"
ds["mass_moment_2"].attrs["units"] = "?"
ds

In [ ]:
data_frac = evaporated_fraction.where(sub_cloud_layer).mean(dim="gridbox") * 100

In [ ]:
# %%
data = -evaporated_mass.where(sub_cloud_layer).sum(dim="gridbox")
data_norm = data / data.max(dim="radius_bins")


top_psd = ds["xi"].sel(gridbox=ds["max_gridbox"]).sel(time=slice(400, None)).mean("time")
top_msd = ds["mass_represented"].sel(gridbox=ds["max_gridbox"]).sel(time=slice(400, None)).mean("time")

In [ ]:
bottom_precipitation = precipitation.sel(gridbox=0).drop_vars("gridbox")
top_skweness = (
    ds["mass_moment_2"]
    .sel(time=slice(400, None))
    .mean("time")
    .sel(gridbox=ds["max_gridbox"])
    .drop_vars("gridbox")
)
top_mass = (
    ds["mass_represented"]
    .sum("radius_bins")
    .sel(time=slice(400, None))
    .mean("time")
    .sel(gridbox=ds["max_gridbox"])
    .drop_vars("gridbox")
)


def get_low_mid_upper(data, coord_name):
    low, mid, upper = (
        data[coord_name].where(data < data.quantile(1 / 3, dim=coord_name), drop=True),
        data[coord_name].where(
            (data >= data.quantile(1 / 3, dim=coord_name))
            & (data < data.quantile(2 / 3, dim=coord_name)),
            drop=True,
        ),
        data[coord_name].where(data >= data.quantile(2 / 3, dim=coord_name), drop=True),
    )
    return low, mid, upper


low_precipitation, mid_precipitation, high_precipitation = get_low_mid_upper(
    bottom_precipitation, "cloud_id"
)
low_skweness, mid_skweness, high_skweness = get_low_mid_upper(top_skweness, "cloud_id")
low_mass, mid_mass, high_mass = get_low_mid_upper(top_mass, "cloud_id")

In [ ]:
# %%

# %%

color_data = bottom_precipitation.expand_dims({"radius_bins": data["radius_bins"]})

my_cmap = mpl.colormaps["plasma_r"]
my_norm = mpl.colors.LogNorm(vmin=color_data.min(), vmax=color_data.max())

colors = my_cmap(np.linspace(0.1, 0.9, 3))

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))


for i, j in enumerate([low_precipitation, mid_precipitation, high_precipitation]):
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        marker=".",
        linestyle="None",
    )
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
    )
    axs[1].plot(
        data["radius_bins"],
        data_norm.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        marker=".",
        linestyle="None",
    )
    axs[1].plot(
        data["radius_bins"],
        data_norm.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
    )


for ax in axs:
    ax.set_xscale("log")
    ax.set_xlim(1e1, None)

# fig.savefig("evaporated_mass.png")

In [ ]:
# %%

# %%

color_data = bottom_precipitation.expand_dims({"radius_bins": data["radius_bins"]})

my_cmap = mpl.colormaps["plasma_r"]
my_norm = mpl.colors.LogNorm(vmin=color_data.min(), vmax=color_data.max())

colors = my_cmap(np.linspace(0.1, 0.9, 3))

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))


for i, j in enumerate([low_skweness, mid_skweness, high_skweness]):
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        # marker = ".",
        # linestyle = "None",
    )
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
        linewidth=2,
    )
    axs[1].plot(
        data["radius_bins"],
        data_norm.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        # marker = ".",
        # linestyle = "None",
    )
    axs[1].plot(
        data["radius_bins"],
        data_norm.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
        linewidth=2,
    )


for ax in axs:
    ax.set_xscale("log")
    ax.set_xlim(1e1, None)

# fig.savefig("evaporated_mass.png")

In [ ]:
# %%

# %%

color_data = bottom_precipitation.expand_dims({"radius_bins": data["radius_bins"]})

my_cmap = mpl.colormaps["plasma_r"]
my_norm = mpl.colors.LogNorm(vmin=color_data.min(), vmax=color_data.max())

colors = my_cmap(np.linspace(0.1, 0.9, 3))

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))


for i, j in enumerate([low_mass, mid_mass, high_mass]):
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        # marker = ".",
        # linestyle = "None",
    )
    axs[0].plot(
        data["radius_bins"],
        data.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
        linewidth=2,
    )
    axs[1].plot(
        data["radius_bins"],
        -data_frac.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.2,
        # marker = ".",
        # linestyle = "None",
    )
    axs[1].plot(
        data["radius_bins"],
        -data_frac.sel(cloud_id=j).mean(dim="cloud_id"),
        c=colors[i],
        linewidth=2,
    )


for ax in axs:
    ax.set_xscale("log")
    ax.set_xlim(1e1, None)

axs[0].set_ylim(None, 0.2e-6)
axs[1].set_yscale("log")
# fig.savefig("evaporated_mass.png")

In [ ]:
# %%

# %%

color_data = bottom_precipitation.expand_dims({"radius_bins": data["radius_bins"]})

my_cmap = mpl.colormaps["plasma_r"]
my_norm = mpl.colors.LogNorm(vmin=color_data.min(), vmax=color_data.max())

colors = my_cmap(np.linspace(0.1, 0.7, 3))

dark_colors = adjust_lightness_array(colors, amount=0.5)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

axs = [ax]

for i, j in enumerate([low_skweness, mid_skweness, high_skweness]):
    axs[0].fill_between(
        top_psd["radius_bins"],
        top_psd.sel(cloud_id=j).quantile(0.33, "cloud_id"),
        top_psd.sel(cloud_id=j).quantile(0.66, "cloud_id"),
        color=colors[i],
        alpha=0.3,
    )

    axs[0].plot(
        top_psd["radius_bins"],
        top_psd.sel(cloud_id=j).T,
        c=colors[i],
        alpha=0.05,
        # marker = ".",
        # linestyle = "None",
    )
    axs[0].plot(
        top_psd["radius_bins"],
        top_psd.sel(cloud_id=j).median(dim="cloud_id"),
        c=dark_colors[i],
        linewidth=2,
    )


for ax in axs:
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(1e-1, None)
axs[0].set_ylim(1e5, 1e7)
# axs[1].set_ylim(1e-3, 1e0)

# fig.savefig("evaporated_mass.png")

(100000.0, 10000000.0)

In [ ]:
def plot_psd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.set_ylim(1e-1, 1e8)
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.scatter(
        ds_rain.radius_array,
        ds_rain.particle_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        ds_lower.particle_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="lower right")

    ax.set_xlabel("radius in m")
    ax.set_ylabel("Occurence in #/m3")

In [ ]:
import datetime
import logging
import os
import sys
import traceback

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml

from sdm_eurec4a import RepositoryPath, conversions, get_git_revision_hash
from sdm_eurec4a.identifications import (
    match_clouds_and_cloudcomposite,
    match_clouds_and_dropsondes,
    select_individual_cloud_by_id,
)

from sdm_eurec4a.reductions import shape_dim_as_dataarray
from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    handler_map_alpha,
    set_custom_rcParams,
)

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation")
print(script_dir)


fig_path = repo_path / "results" / script_dir.relative_to(repo_path)
print(fig_path)

fig_path.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/presentation


In [ ]:
identified_clouds_path = repo_path / Path(
    "data/observation/cloud_composite/processed/identified_clouds/identified_clusters_rain_mask_5.nc"
)
cloud_composite_path = repo_path / Path(
    f"data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
drop_sondes_path = repo_path / Path("data/observation/dropsonde/processed/drop_sondes.nc")
distance_path = repo_path / Path(
    "data/observation/combined/distance_relations/distance_dropsondes_identified_clusters_rain_mask_5.nc"
)
satellite_path = repo_path / Path(
    "data/observation/satellite/test/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20200261630156_BARBADOS-2KM-FD.level2.nc"
)

In [ ]:
# %%

distance_IC_DS = xr.open_dataset(distance_path)

cloud_composite = xr.open_dataset(cloud_composite_path)

drop_sondes = xr.open_dataset(drop_sondes_path)

drop_sondes = drop_sondes.where(drop_sondes.alt <= 1600, drop=True)

# add relative humidity to the dataset

drop_sondes["relative_humidity"] = conversions.relative_humidity_from_tps(
    temperature=drop_sondes["air_temperature"],
    pressure=drop_sondes["pressure"],
    specific_humidity=drop_sondes["specific_humidity"],
)


# Add radii and altitude arrays to datasets

cloud_composite["radius_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="radius"
)
cloud_composite["time_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="time"
)
drop_sondes["alt_array"] = shape_dim_as_dataarray(da=drop_sondes["air_temperature"], output_dim="alt")

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

rename dataarray
rename dataarray
rename dataarray


In [ ]:
low_precip_clouds = (
    identified_clouds.swap_dims({"time": "cloud_id"})
    .sel(cloud_id=low_skweness)
    .swap_dims({"cloud_id": "time"})
)
mid_precip_clouds = (
    identified_clouds.swap_dims({"time": "cloud_id"})
    .sel(cloud_id=mid_skweness)
    .swap_dims({"cloud_id": "time"})
)
high_precip_clouds = (
    identified_clouds.swap_dims({"time": "cloud_id"})
    .sel(cloud_id=high_skweness)
    .swap_dims({"cloud_id": "time"})
)

In [ ]:
low_cc = match_clouds_and_cloudcomposite(low_precip_clouds, cloud_composite)
mid_cc = match_clouds_and_cloudcomposite(mid_precip_clouds, cloud_composite)
high_cc = match_clouds_and_cloudcomposite(high_precip_clouds, cloud_composite)

In [ ]:
fig, axs = plt.subplots(4, 1, sharex=True, sharey=True, figsize=(10, 15))

plot_psd_scatter(fig, axs[0], low_cc)
axs[0].plot(low_cc["radius"], low_cc["particle_size_distribution"].mean("time"))
plot_psd_scatter(fig, axs[1], mid_cc)
axs[1].plot(mid_cc["radius"], mid_cc["particle_size_distribution"].mean("time"))
plot_psd_scatter(fig, axs[2], high_cc)
axs[2].plot(high_cc["radius"], high_cc["particle_size_distribution"].mean("time"))

for d, name in zip([low_cc, mid_cc, high_cc], ["low", "mid", "high"]):
    axs[3].plot(
        d["radius"],
        d["particle_size_distribution"].where(d["particle_size_distribution"] != 0).mean("time"),
        label=name,
    )
axs[3].legend()

In [ ]:
data = (
    ds["xi"]
    .sel(gridbox=ds["max_gridbox"])
    .sel(time=slice(400, None))
    .mean("time")
    .sel(cloud_id=low_skweness)
)